In [1]:
import pandas as pd
import numpy as np

#### This notebook extracts all duplicates from the original dataset for a given year. This code was run for every year to generate the duplicate dataset.

In [27]:
frames = []
chunkIterator = pd.read_csv("/Users/ServantezTech/Desktop/chicago_parking_tickets.csv", chunksize=100000, dtype=object)

for chunk in chunkIterator:
    
    frame = chunk[chunk["year"] == "2018"]
    frames.append(frame)

data = pd.concat(frames)

print("Phase 1 Complete")

hashDict = dict()

for index, row in data.iterrows():
        
    rawDate = row["issue_date"]
    date = rawDate.split(" ", 2)[0]
    ticketNumber = row["ticket_number"]
    licensePlateNum = row["license_plate_number"]
    code = row["violation_code"]
        
    uuid = licensePlateNum + "&" + ticketNumber
    key = licensePlateNum + date + code
        
    if key in hashDict:
        hashDict[key].append(uuid)
    else:
        hashDict[key] = [uuid]

print("Phase 2 Complete")

duplicateIDs = []

for key, value in hashDict.items():
    
    if len(value) > 1:
        duplicateIDs = duplicateIDs + value

hashDict.clear()
duplicateDict = dict() #Not sure if we need both values to identify

for uuid in duplicateIDs:
    
    segments = uuid.split("&", 2)
    duplicateDict[segments[1]] = segments[0]
    
ticketNumbers = list(duplicateDict.keys())

print("Phase 3 Complete")

segments = []
chunks = pd.read_csv("/Users/ServantezTech/Desktop/chicago_parking_tickets.csv", chunksize=100000, dtype=object)

for chunk in chunks:
    
    segment = chunk[chunk["ticket_number"].isin(ticketNumbers)]
    segments.append(segment)

new = pd.concat(segments)
old = pd.read_csv("/Users/ServantezTech/Desktop/duplicates.csv", dtype=object)
duplicates = pd.concat([old, new])

duplicates.to_csv("/Users/ServantezTech/Desktop/duplicates.csv", index=False)
print("Phase 4 Complete")

Phase 1 Complete
Phase 2 Complete
Phase 3 Complete
Phase 4 Complete


In [28]:
preview = pd.read_csv("/Users/ServantezTech/Desktop/duplicates.csv", delimiter=',', dtype = object)
preview.shape

(3265162, 32)

In [30]:
prev = preview[preview["year"] == "2018"]
prev.head()

,ticket_number,issue_date,violation_location,license_plate_number,license_plate_state,license_plate_type,zipcode,violation_code,violation_description,unit,...,normalized_address,year,month,hour,ward,geocode_accuracy,geocode_accuracy_type,geocoded_address,geocoded_lng,geocoded_lat
3229277,68783146,2018-01-01 08:48:00,2300 N PARKSIDE,c2083069204047c50c2792d39043ef36e1e3eacefdd137...,IL,PAS,NaN,0964150B,PARKING/STANDING PROHIBITED ANYTIME,25,...,"2300 n parkside, Chicago, IL",2018,1,8,NaN,NaN,NaN,NaN,NaN,NaN
3229278,69061739,2018-01-01 09:10:00,4319 W ADAMS,520eb0c8da9a4d4185978b7748ed12d6687527ab2c597b...,IL,PAS,608041609,0976160B,EXPIRED PLATE OR TEMPORARY REGISTRATION,11,...,"4300 w adams, Chicago, IL",2018,1,9,29,1,range_interpolation,"4300 W Adams Blvd, Chicago, IL 60644",-87.765004,41.877934
3229279,68949779,2018-01-01 11:14:00,921 E 47THST,c2083069204047c50c2792d39043ef36e1e3eacefdd137...,IL,PAS,NaN,0976160A,MISSING/NONCOMPLIANT FRONT AND/OR REAR PLATE,2,...,"900 e 47thst, Chicago, IL",2018,1,11,4,1,rooftop,"900 E 47th St, Chicago, IL 60653",-87.603613,41.809789
3229280,68828855,2018-01-01 12:12:00,5333 S JUSTINE,28cdcdfd7161e28a0b4119b857070ee6ae1b2dda0d13cf...,IL,PAS,606095813,0976160B,EXPIRED PLATE OR TEMPORARY REGISTRATION,9,...,"5300 s justine, Chicago, IL",2018,1,24,16,1,rooftop,"5300 S Justine St, Chicago, IL 60609",-87.663818,41.797568
3229281,68715731,2018-01-01 15:45:00,4317 W ADAMS,520eb0c8da9a4d4185978b7748ed12d6687527ab2c597b...,IL,PAS,608041609,0976160B,EXPIRED PLATE OR TEMPORARY REGISTRATION,11,...,"4300 w adams, Chicago, IL",2018,1,15,29,1,range_interpolation,"4300 W Adams Blvd, Chicago, IL 60644",-87.765004,41.877934
